# Exercícios Pandas Parte 2 # 

In [8]:
import pandas as pd
import numpy as np


## Exercício 1 ##
Olhando apenas para o grupo de divorciados, qual o percentual deles que é empreendedor (entrepreneur)? E estudantes?.</p>


In [9]:
bank_df= pd.read_csv('bank-full.csv', sep=';')
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


Resolução

<p> Primeiro, vou verificar quantos tipos únicos temos de jobs e estados cívis.</p>

In [10]:
#verificando tipos únicos
bank_df.job.unique()
bank_df.marital.unique()

array(['married', 'single', 'divorced'], dtype=object)

<p> Para job temos as seguintes categorias: </p>
<ul> management', 'technician', 'entrepreneur', 'blue-collar',
       'unknown', 'retired', 'admin.', 'services', 'self-employed',
       'unemployed', 'housemaid', 'student </ul>
<p> E para marital temos as seguintes categorias:
<ul>['married', 'single', 'divorced'] </ul>

In [11]:
#filtrando divorciados e estudantes que sao divorciados
quantidade= bank_df.query('job in[ "entrepreneur", "student"] and marital == "divorced"')

#contando a quantidade de pessoas que são divorciadas e ou estudantes ou empresarios
qtd= quantidade.shape[0]

#quantidade de divorciados
divorciados= bank_df.query('marital == "divorced"')
qtde_divorciados= divorciados.shape[0]

#percentual de divorciados que são estudantes ou empresários
percentual= (qtd/qtde_divorciados) *100

print(f'O número de pessoas que são divorciadas e são ou estudante ou empresários é de {qtd}')
print(f' O total dos divorciados é de {qtde_divorciados}')
print(f' E o percentual de divorciados que são divorciados e ao mesmo tempo empresários ou estudantes é de {percentual:.2f} %')

#obtendo o percentual

O número de pessoas que são divorciadas e são ou estudante ou empresários é de 185
 O total dos divorciados é de 5207
 E o percentual de divorciados que são divorciados e ao mesmo tempo empresários ou estudantes é de 3.55 %


Talvez tenha melhor forma de fazer isso, mas seguimos em frente. O importante é continuar.
</details>

## Exercício 2 ##
 O seu chefe decidiu criar duas categorias, Premium e Gold, para focar nos clientes mais promissores. Para a categoria Premium, entrarão os que possuem saldo (balance) nos top 10%. Ou seja, o grupo acima do percentil 90. Já o grupo Gold, ficarão os que possuem saldo nos top 20%, mas abaixo do grupo Premium. Ou seja, do p80 até o último do p90. Verifique quantos clientes possuem no grupo Premium e no Gold.

Primeiramente, iremos calcular o percentil de 90% e 80% do balance:

grupo premium: 10 % com mais balance, percentil de 90% 
grupo gold: percentil de 80% até 90%

categoria defina pelo balance

In [12]:
# definindo percentil de 80%
percent_80= bank_df['balance'].quantile(0.80)

#definindo percentil de 90%
percent_90= bank_df['balance'].quantile(0.90)

percent_80
percent_90

#definindo intervalos para o cut

bins= [bank_df['balance'].min(), percent_80, percent_90, bank_df['balance'].max()]
labels=['gold', 'premium', 'top 10%']

#categorizando os dados

bank_df['categoria']=pd.cut(bank_df['balance'], bins=bins, labels=labels, include_lowest= True)

#contando quantos clientes estão em cada grupo
contagem= bank_df['categoria'].value_counts()

print(bank_df)
print(f"O número de clientes na categoria Gold : {contagem.get('gold',0)}")
print(f"O número de clientes na categoria Premium: {contagem.get('premium',0)}")

       age           job   marital  education default  balance housing loan  \
0       58    management   married   tertiary      no     2143     yes   no   
1       44    technician    single  secondary      no       29     yes   no   
2       33  entrepreneur   married  secondary      no        2     yes  yes   
3       47   blue-collar   married    unknown      no     1506     yes   no   
4       33       unknown    single    unknown      no        1      no   no   
...    ...           ...       ...        ...     ...      ...     ...  ...   
45206   51    technician   married   tertiary      no      825      no   no   
45207   71       retired  divorced    primary      no     1729      no   no   
45208   72       retired   married  secondary      no     5715      no   no   
45209   57   blue-collar   married  secondary      no      668      no   no   
45210   37  entrepreneur   married  secondary      no     2971      no   no   

         contact  day month  duration  campaign  pd

Versão 2
Primeiro, vamos calcular também o percentil de 90 e 80


In [13]:
#Calculando o percentil de 90 e 80

percentiu_90= np.percentile(bank_df['balance'],90)
percentiu_80= np.percentile(bank_df['balance'],80)

#Criando coluna para categoria Premium

bank_df['CATEGORIA']= np.where(bank_df['balance'] >= percentiu_90, 'Premium',
                               np.where(bank_df['balance']>= percentiu_80, 'Gold', 'Outros'))

#np Where cria novas tabelas com base nas condições

#Contando os Clientes em cada categoria

contagem_categoria= bank_df['CATEGORIA'].value_counts()
print(contagem_categoria)

Outros     36162
Gold        4525
Premium     4524
Name: CATEGORIA, dtype: int64


A primeira solução é mais flexível e a segunda é mais fácil e intuitiva.

## Exercício 3 ##

3) Seu chefe também quer saber se existe diferença nos percentuais vistos no exercício (1) para o grupo Premium. Verifique como a tabela cruzada da população se diferencia do grupo Premium. Existe diferença? Qual sua hipótese para que exista essa diferença?

In [20]:
#Selecionando apenas os premium na coluna de categoria
premium_df = bank_df[bank_df['CATEGORIA'] == 'Premium']
premium_df

#Tabela cruzada para população total
crosstab_total= pd.crosstab(bank_df['job'], bank_df['marital'], normalize= 'index')

#tabela cruzada de job apenas para categoria premium
crosstab_premium= pd.crosstab(premium_df['job'], premium_df['marital'], normalize= 'index')

#comparando percentuais
print(crosstab_total)
print(crosstab_premium)

marital        divorced   married    single
job                                        
admin.         0.145040  0.520789  0.334171
blue-collar    0.077065  0.715988  0.206946
entrepreneur   0.120377  0.719570  0.160054
housemaid      0.148387  0.735484  0.116129
management     0.117467  0.570945  0.311588
retired        0.187721  0.764576  0.047703
self-employed  0.088664  0.628879  0.282457
services       0.132162  0.579442  0.288397
student        0.006397  0.057569  0.936034
technician     0.121759  0.533368  0.344873
unemployed     0.131236  0.561013  0.307751
unknown        0.059028  0.704861  0.236111
marital        divorced   married    single
job                                        
admin.         0.100478  0.612440  0.287081
blue-collar    0.056604  0.723720  0.219677
entrepreneur   0.083916  0.825175  0.090909
housemaid      0.171642  0.701493  0.126866
management     0.104361  0.598131  0.297508
retired        0.160237  0.818991  0.020772
self-employed  0.134409  0.61828

A diferença encontrada foi de que entre o grupo de premiuns, o percentual de empresários e que são casados, são bem superiores ao restante da população. Há também mais empresários casados do que solteiros. Acredito que isso seja esperado, por talvez serem homens com mais idade, já vivendo o matrimônio.

### exercício 4 ###

4) Para a apresentação final, você decide criar uma tabela com os 10 clientes que possuem maior saldo (balance). Como a tabela é grande demais, você resolve apresentar apenas as informações relevantes: age, job, marital, education, default, loan, duration. Além disso, você decide destacar clientes que foram mais e menos atenciosos, pois são os que parecem ter maior e menor potencial, respectivamente. Para isso, destaque na tabela, de amarelo, o cliente de maior duration. Destaque, de vermelho, o que tem menor duration.


In [36]:
#selecionando as tabelas que quero do dataframe
select_tab= ['balance','age', 'job', 'marital', 'education', 'default', 'loan', 'duration']

#criando um dataframe filtrado
filtered_bankdf= bank_df[select_tab]

#selecionando os top 10 com maiorees balances
top_10balance= filtered_bankdf.nlargest(10, 'balance')
print(top_10balance)

# Função para aplicar cores aos valores máximos e mínimos na coluna 'duration'
def highlight_max_min(data):
    styles = pd.DataFrame('', index=data.index, columns=data.columns)
    max_val = data['duration'].max()
    min_val = data['duration'].min()
    
    # Aplicando o estilo
    styles['duration'] = ['background-color: yellow' if v == max_val else 'background-color: red' if v == min_val else '' for v in data['duration']]
    
    return styles

# Aplicando a função ao DataFrame
styled_df = top_10balance.style.apply(highlight_max_min, axis=None)

# Exibindo o DataFrame estilizado
styled_df

       balance  age           job   marital  education default loan  duration
39989   102127   51    management    single   tertiary      no   no        90
26227    98417   59    management   married   tertiary      no   no       145
42558    81204   84       retired   married  secondary      no   no       679
43393    81204   84       retired   married  secondary      no   no       390
41693    71188   60       retired   married    primary      no   no       205
19785    66721   56    management  divorced   tertiary      no   no       442
21192    66653   52   blue-collar   married    primary      no   no       109
19420    64343   59        admin.   married    unknown      no   no        45
41374    59649   32  entrepreneur    single   tertiary      no   no        69
12926    58932   56   blue-collar   married  secondary      no   no       339


,balance,age,job,marital,education,default,loan,duration
39989,102127,51,management,single,tertiary,no,no,90
26227,98417,59,management,married,tertiary,no,no,145
42558,81204,84,retired,married,secondary,no,no,679
43393,81204,84,retired,married,secondary,no,no,390
41693,71188,60,retired,married,primary,no,no,205
19785,66721,56,management,divorced,tertiary,no,no,442
21192,66653,52,blue-collar,married,primary,no,no,109
19420,64343,59,admin.,married,unknown,no,no,45
41374,59649,32,entrepreneur,single,tertiary,no,no,69
12926,58932,56,blue-collar,married,secondary,no,no,339


### Exercício 5 ###